In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

In [4]:
# create a tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers a and b and returns the product."""
    return a * b

In [5]:
print(multiply.invoke({ "a": 2, "b": 3}))

6


In [11]:
# create a llm 
chat_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash"
)

In [10]:
# this is how we can bind the tool to the llm
llm_with_tools = chat_model.bind_tools([multiply])

In [12]:
llm_with_tools.invoke("hi how are you ?")
# here llm knows that he dont need to use the tool

AIMessage(content="I'm doing well, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-fe59e16e-cb35-418d-87a2-2850dbf7c631-0', usage_metadata={'input_tokens': 25, 'output_tokens': 17, 'total_tokens': 42, 'input_token_details': {'cache_read': 0}})

In [34]:
query = HumanMessage("what is 2 * 3 ?")
messages = [query]

In [35]:
messages

[HumanMessage(content='what is 2 * 3 ?', additional_kwargs={}, response_metadata={})]

In [25]:
# since we have created a tool to multiply two numbers and bind it to the llm, so llm will automatically use the tool to multiply two numbers
result = llm_with_tools.invoke(messages[0].content)
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 2.0, 'b': 3.0},
 'id': 'd52de42a-2048-4ba8-abfc-b358e53bdc3d',
 'type': 'tool_call'}

In [36]:
messages.append(result)
messages

[HumanMessage(content='what is 2 * 3 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-d245e69d-bd0e-477d-bb69-4698a058f48f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': 'd52de42a-2048-4ba8-abfc-b358e53bdc3d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 28, 'output_tokens': 3, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}})]

## NOTE
- llm will never call the tool directly, it will just suggest the tool to use with the input parameters.
- the tool will be either called by the user or by the created agent.
---
> here we will learn how to use the tool suggested by the llm.

In [39]:
tool_result = multiply.invoke(result.tool_calls[0])
tool_result

ToolMessage(content='6', name='multiply', tool_call_id='d52de42a-2048-4ba8-abfc-b358e53bdc3d')

In [38]:
messages.append(tool_result)
messages

[HumanMessage(content='what is 2 * 3 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-d245e69d-bd0e-477d-bb69-4698a058f48f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': 'd52de42a-2048-4ba8-abfc-b358e53bdc3d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 28, 'output_tokens': 3, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='6', name='multiply', tool_call_id='d52de42a-2048-4ba8-abfc-b358e53bdc3d')]

In [40]:
result = llm_with_tools.invoke(messages)

In [42]:
result.content

'6'